# rcsbsearchapi quickstart

This notebook contains examples from the rcsbsearchapi [quickstart](https://rcsbsearchapi.readthedocs.io/en/latest/quickstart.html)

In [ ]:
from rcsbsearchapi.search import TextQuery
from rcsbsearchapi import rcsb_attributes as attrs

## Operator syntax

Here is an example from the RCSB Search API page, using the operator syntax. This query finds symmetric dimers having a twofold rotation with the DNA-binding domain of a heat-shock transcription factor.

Note the use of standard comparison operators (`==`, `>` etc) for rcsb attributes and set operators for combining queries.

In [ ]:
# Create terminals for each query
q1 = TextQuery("heat-shock transcription factor")
q2 = attrs.rcsb_struct_symmetry.symbol == "C2"
q3 = attrs.rcsb_struct_symmetry.kind == "Global Symmetry"
q4 = attrs.rcsb_entry_info.polymer_entity_count_DNA >= 1

# combined using bitwise operators (&, |, ~, etc)
query = q1 & q2 & q3 & q4 # AND of all queries

# Call the query to execute it
for assemblyid in query("assembly"):
    print(assemblyid)


Attribute names can be found in the [RCSB schema](http://search.rcsb.org/rcsbsearch/v2/metadata/schema). They can also be found via tab completion, or by iterating:

In [ ]:
[a.attribute for a in attrs if "authors" in a.attribute]

## Fluent syntax

Here is the same example using the fluent syntax:

In [ ]:
# Start with a Attr or TextQuery, then add terms
results = TextQuery('"heat-shock transcription factor"') \
    .and_("rcsb_struct_symmetry.symbol").exact_match("C2") \
    .and_("rcsb_struct_symmetry.kind").exact_match("Global Symmetry") \
    .and_("rcsb_entry_info.polymer_entity_count_DNA").greater_or_equal(1) \
    .exec("assembly")

# Exec produces an iterator of IDs
for assemblyid in results:
    print(assemblyid)

For a more practical example, see the [Covid-19 notebook](covid.ipynb)